# torch.nn以及torch.optim

## 1 torch.nn
### 直观理解
* 在keras、TFLearn等高级的深度学习集成库中，将许多计算进行更加高级的封装
* torch.nn包内就是类似这种高级封装，里面包括了卷积，池化，RNN等计算,以及其他如Loss计算，可以把torch.nn包内的各个类想象成神经网络的一层
* 创建一个torch.nn中的层后，对其输入Variable，将得到输出的Variable，具体例子见下示例

### 作用
* 这种方式将网络高度模块化，使得网络的每个运算都像积木一样一层又一层叠加。
* 它的作用包括，能够接受一个输入，并得到一个输出(forward操作)。同时，他保存了该层学习的参数，以及其他的一些状态变量

### 实现机制
* torch.nn包里面的大部分类都是继承了父类torch.nn.Modules，即本质上所有torch.nn中的层（如卷积、池化）都是torch.nn.Modules
* torch.nn.Modules可以简单认为是网络一层，包括该层的参数，以及一些操作，如forward，调用参数等
* 同样地，它的作用包括，能够接受一个输入，并得到一个输出(forward操作)。同时，他保存了该层学习的参数，以及其他的一些状态变量。并实现各类功能的方法，如返回Modules的参数等
* 因此可以把torch.nn以及torch.nn.Modules看做是一个定义好的小网络（包括其参数），给该网络一个输入，其将得到一个输出。输入输出都是Variable
* 更多Modules具体机制以及如何重写将在后面介绍

### torch.nn示例

In [30]:
-

模型：  Sequential (
  (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
  (2): ReLU ()
  (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
)
模型参数:  <bound method Sequential.parameters of Sequential (
  (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
  (2): ReLU ()
  (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
)>
卷积层参数个数：  torch.Size([16, 1, 5, 5])
torch.Size([1, 16, 5, 5])
旧的参数值：  
(0 ,.,.) = 
  0.0818  0.1192  0.1884  0.0289  0.1282
 -0.0269 -0.1857 -0.1634 -0.0330 -0.1028
  0.1419  0.1765 -0.0709  0.0598  0.0435
 -0.1624 -0.0440 -0.0329  0.0813  0.0940
  0.1298  0.1757  0.1506  0.0168 -0.0209
[torch.FloatTensor of size 1x5x5]

新的参数值：  
(0 ,.,.) = 
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
[torch.FloatTensor of size 1x5x5]

MSELoss (
)


## torch.optim
* 可以直接使用optim来针对制定的参数进行优化
* 每次需先计算出梯度即backward后，才能进行优化

### torch.optim 示例

In [ ]:
# 创建一个optim，并制定对应的参数
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# forward 然后 backward 得到梯度
# y = model(x)
# y.backward()

# 将对应参数梯度置0
optimizer.zero_grad()

# 更新参数
optimizer.step()

## 例子

In [6]:
# Code in file nn/two_layer_net_optim.py
import torch
from torch.autograd import Variable

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs, and wrap them in Variables.
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
          torch.nn.Linear(D_in, H),
          torch.nn.ReLU(),
          torch.nn.Linear(H, D_out),
        )
loss_fn = torch.nn.MSELoss(size_average=False)

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Variables it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(20000):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x).mean()

    # Compute and print loss.
#     loss = loss_fn(y_pred, y)
#     print(t, loss.data[0])

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable weights
    # of the model)
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model parameters
#     loss.backward()
    y_pred.backward()
    if t % 100 == 0:
        print(y_pred.data[0])

    # Calling the step function on an Optimizer makes an update to its parameters
    optimizer.step()

0.0760311111808
-1.43488168716
-4.80305242538
-10.8855791092
-19.8142986298
-31.4813365936
-45.7385292053
-62.4501876831
-81.5586395264
-103.017082214
-126.720848083
-152.583770752
-180.485992432
-210.315795898
-242.073486328
-275.739654541
-311.207519531
-348.420196533
-387.39932251
-428.143310547
-470.586273193
-514.656982422
-560.32623291
-607.590026855
-656.414428711
-706.7265625
-758.504211426
-811.717590332
-866.39050293
-922.446472168
-979.888793945
-1038.74121094
-1098.98596191
-1160.60217285
-1223.51220703
-1287.70996094
-1353.17236328
-1419.91833496
-1488.00378418


KeyboardInterrupt: 